In [2]:
import requests
import json
import pandas as pd

# response = requests.get("https://ubcgrades.com/api/v3/grades/UBCV/2022S")
# data = response.json()
# df = pd.json_normalize(data)
# df
### 
yearSessionsResponse = requests.get("https://ubcgrades.com/api/v3/yearsessions/UBCV")
yearSessions = yearSessionsResponse.json()
campuses = ["UBCV", "UBCO"]
data = []
urlCounter = 0
for campus in campuses:
    for yearSession in yearSessions:
        url = f"https://ubcgrades.com/api/v3/grades/{campus}/{yearSession}"
        urlCounter += 1
        response = requests.get(url)
        jsonData = response.json()
        data.extend(jsonData) # TODO: add yearSession to object
df = pd.json_normalize(data)
# df
print(df.shape)
print(urlCounter)

dfbackup = df.copy()
print(df)


(26139, 29)
10
       average campus course  \
0         93.9   UBCV    390   
1         86.7   UBCV    327   
2         84.6   UBCV    327   
3         86.8   UBCV    327   
4         84.6   UBCV    328   
...        ...    ...    ...   
26134     75.4   UBCO    150   
26135     96.4   UBCO    399   
26136     96.4   UBCO    399   
26137     85.5   UBCO    480   
26138     85.5   UBCO    480   

                                            course_title detail  \
0                   Asian Migrations in a Global Context      A   
1                                        Teaching Adults          
2                                        Teaching Adults          
3                                        Teaching Adults          
4                        Institutions of Adult Education          
...                                                  ...    ...   
26134        Introduction to Intercultural Communication          
26135                Special Topics in World Literatures      O 

In [3]:
columns = [col for col in df.columns]
columns

['average',
 'campus',
 'course',
 'course_title',
 'detail',
 'educators',
 'faculty_title',
 'high',
 'low',
 'median',
 'percentile_25',
 'percentile_75',
 'reported',
 'section',
 'session',
 'subject',
 'subject_title',
 'year',
 'grades.50-54%',
 'grades.55-59%',
 'grades.60-63%',
 'grades.64-67%',
 'grades.68-71%',
 'grades.72-75%',
 'grades.76-79%',
 'grades.80-84%',
 'grades.85-89%',
 'grades.90-100%',
 'grades.<50%']

# Import grades into dataframe
API: https://ubcgrades.com/api/v3/grades/UBCV/2022S
Options: `/grades/campus/yearsession` 
- campus: `UBCV` or `UBCO`
Note: '[V3] carries data from 2019W onward...Some endpoints are not available in v3, and v2 should be used instead.'

In [13]:
# df.to_csv('v3Raw2.csv') # includes index (first) column

In [5]:
# SELECT campus, subject, course where average > 90
# questions: what courses in 3rd and 4th year can I take that have the highest average?
# what courses have the lowest fail rate? (<50 or <55%)
# how do grades compare between sections of the same class?
# how do grades compare between profs over the years/classes?
# does class size influence grades?
# what subjects (CPSC, MATH, etc) have the highest averages?
# what MATH 1xx courses have the highest and lowest averages?

#!pip install pandasql

In [6]:
# !pip install sqldf
import sqldf
# note seaborn is an alternative to plotly?! https://seaborn.pydata.org/
# https://medium.com/@mohsin.shaikh324/matplotlib-vs-seaborn-vs-plotly-a-comparative-guide-c99a0059c09f
# maybe its why plotly + dash seem more business focused, not uni. cause uni are more research paper oriented and probably seaborn is more commonly used
# print(sqldf('''# SELECT campus, subject, course WHERE average > 90'''))

query = """
SELECT campus, subject, course, average, year
FROM df
WHERE average >= 90  AND year >= 2020 AND course <500
"""
df_view = sqldf.run(query)
df_view

,campus,subject,course,average,year
0,UBCV,ACAM,390,93.9,2021
1,UBCV,ADHE,328,91.4,2021
2,UBCV,ADHE,330,90.6,2021
3,UBCV,APBI,497,91.2,2021
4,UBCV,ARCL,306,91.6,2021
...,...,...,...,...,...
1168,UBCO,VISA,460,91.7,2023
1169,UBCO,VISA,460,100.0,2023
1170,UBCO,VISA,460,100.0,2023
1171,UBCO,WRLD,399,96.4,2023


In [7]:
# !pip install pandasai

In [8]:
# Pandas and OpenAI
# pandasai examples https://github.com/gventuri/pandas-ai/tree/main/examples * 
from pandasai import SmartDataframe
from pandasai.llm import OpenAI
import os #!pip install python-dotenv
import openai # do i need this?

temp = os.getenv("OPENAI_API_KEY")
llm = OpenAI(api_token=temp) # note: set OPENAI_API_KEY=__ in .env file
smartdf = SmartDataframe(df, config={"llm": llm})
response = smartdf.chat("What course has the highest average? Give information about year, course, subject, and the average itself.")
print(response)

The course with the highest average is Asian Migrations in a Global Context. The average is 80.97555401460937.


In [9]:
response = smartdf.chat("What course has the highest average? Give information about year, course, subject, and the average itself.")
print(response)

The course with the highest average is Asian Migrations in a Global Context. The average is 80.97555401460937.


In [10]:
# !pip install python-dotenv

In [11]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
# import pandas as pd

# df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminder_unfiltered.csv')

app = Dash(__name__)

app.layout = html.Div([
    html.H1(children='Title of Dash App', style={'textAlign':'center'}),
    dcc.Dropdown(df.year.unique(), '2020', id='dropdown-selection'),
    dcc.Graph(id='graph-content')
])

@callback(
    Output('graph-content', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_graph(value):
    dff = df[df.faculty_title==value]
    return px.line(dff, x='course', y='average')

if __name__ == '__main__':
    app.run(debug=True)
